In [1]:
import glob
import xml.etree.ElementTree as ET
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

In [4]:
files = glob.glob("../../../datasets/Allerhande_1978-2009/**/*.xml", recursive=True)

In [5]:
d = []
for file in tqdm(files):
    if file.endswith('xml'):
        filename = os.path.splitext(os.path.basename(file))[0]
        issue = filename.split('_')[0]
        page_nr = filename.split('_')[1]
        doc = ET.parse(file)
        for text in doc.findall(".//{http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml}text"):  
            # calculate average OCR confidence per article
            quality = []
            for char in text.findall(".//{http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml}charParams"):
                if char.get("charConfidence") is not None:
                    quality.append(int(char.get("charConfidence")))
            avg_quality = np.mean(quality)
            if avg_quality != 'NaN':
                # extract texts per line
                words = []
                for line in text.findall(".//{http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml}line"):
                    characters = []
                    try:
                        for char in line.findall(".//{http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml}charParams"):
                            characters.append(char.text)
                        word = ''.join(characters)
                        # fix end of line 
                        if word.endswith('-'):
                            word = word[:-1]
                        else:
                            word = word + ' '
                        words.append(word)
                    except Exception as e:
                        pass
                text_ = ''.join(words)
                d.append({'page_nr': page_nr, 'issue': issue,
                      'avg_quality': avg_quality, 'content': text_})
            else:
                pass


df = pd.DataFrame(d)

  0%|          | 0/42896 [00:00<?, ?it/s]/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 42896/42896 [09:42<00:00, 73.59it/s] 


In [7]:
df.dropna(inplace=True)

In [13]:
distributions = pd.read_csv('../data/issues.csv')

In [18]:
df_merge = df.merge(distributions, left_on='issue', right_on='ID')

In [23]:
df_merge.drop('ID', axis=1, inplace=True)

In [33]:
df_merge.sort_values(['year', 'month', 'day', 'page_nr'])

,avg_quality,content,issue,page_nr,total_pages,year,month,day,comment
307759,97.620879,"AITs eigen chocolademerk in melk,puur, extra b...",SAH010000001,0002,32,1979.0,12,1.0,NaN
307725,99.327968,Traditiegetrouw bent u gewend dat Albert Heijn...,SAH010000001,0003,32,1979.0,12,1.0,NaN
307726,99.400000,journalisten. Deze vooraanstaande specialisten...,SAH010000001,0003,32,1979.0,12,1.0,NaN
307727,94.000000,-Inhoud,SAH010000001,0003,32,1979.0,12,1.0,NaN
307728,98.987654,"5 Vijf kerstmenu's 6 Klassiek menu 8 ""Onder on...",SAH010000001,0003,32,1979.0,12,1.0,NaN
307729,99.442308,20 De fijnste wijnen 23 Kaarsen in kerstsfeer ...,SAH010000001,0003,32,1979.0,12,1.0,NaN
307672,97.802632,"In een schelpje,of in een knapperig bladerdeeg...",SAH010000001,0004,32,1979.0,12,1.0,NaN
307673,99.500000,UNOX,SAH010000001,0004,32,1979.0,12,1.0,NaN
307674,98.000000,ragout,SAH010000001,0004,32,1979.0,12,1.0,NaN
307675,98.000000,4,SAH010000001,0004,32,1979.0,12,1.0,NaN


In [35]:
df_merge.to_csv('../data/allerhande_1979_2001.csv', index=False)